In [ ]:
import pandas as pd
import numpy as np
import gpxpy
import gpxpy.gpx
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import tree
from sklearn.metrics import accuracy_score

In [ ]:
def Load_gpx(filename):
    gpx_file = open('{}.gpx'.format(filename), 'r') 
    gpx = gpxpy.parse(gpx_file)
    return gpx

In [ ]:
#gpx = Load_gpx('Lunch_Run')
#gpx = Load_gpx('Afternoon_Ride')
#gpx = Load_gpx('Evening_Ride')
gpx = Load_gpx('Evening_Ride_heart_rate')

In [ ]:
from lxml import etree
import xml.etree.ElementTree as ET

In [ ]:
tree = ET.parse('Evening_Ride_heart_rate.gpx')
root = tree.getroot()
print(root)

In [ ]:
root = tree.getroot()

In [ ]:
root[1]
string = ET.tostring(root[1])
#string
#print string.split("hr>")[5]

In [ ]:
import re
#r = re.compile('hr>(.*?)</ns')
#m = r.search(string)
#if m:
#    print m
    #lyrics = m.group(1)

In [ ]:
heart_rate = re.findall('hr>(.*?)</ns' ,string)
len(heart_rate)

In [ ]:
def Extract_speed(gpx):
    speed = []

    for track in gpx.tracks:
        for segment in track.segments:
            for point_no, point in enumerate(segment.points):
                if point.speed != None:
                    print "Speed=", point.speed
                elif point_no > 0:
                    #print "Calculated speed=", point.speed_between(segment.points[point_no - 1])
                    speed.append(point.speed_between(segment.points[point_no - 1]))
    return speed
    
def Extract_elevation(gpx):    
    elevation = []
    for track in gpx.tracks:
        for segment in track.segments:
            
            for point in segment.points:
                #print(segment.points)
                elevation.append(point.elevation)
    #print segment.points
    return elevation


#def Extract_heart(gpx):
 #   heart = []
  #  for track in gpx.tracks:
   #     for segment in track.segments:
    #        for point in segment.points:
     #           for extension in point.extensions:
      #              for TrackPointExtension in extension.'gpxtpx:TrackPointExtensions':
       #                 #print extension.TrackPointExtensions
        #                    print point.extensions
         #               #heart.append(extension.hr)
    
   #return extension.

In [ ]:
# Input data for the machine learning
speed = Extract_speed(gpx)
elevation = Extract_elevation(gpx)
#heart = Extract_heart(gpx)
len(elevation)
len(speed)

In [ ]:
# take elevation and manipulate
gradient = []
for value in range(1,len(elevation)):
    gradient.append(elevation[value]-elevation[value-1])

#Test and Train
speed_array = np.asarray(speed)
gradient_array = np.asarray(gradient)
heart_array = np.asarray(heart_rate)
heart_array = heart_array[:-1]

speed_array
heart_array = heart_array.astype(np.float)

In [ ]:
len(speed_array)

In [ ]:
plt.plot(gradient_array, 'ro')
plt.show()
plt.plot(speed_array, 'bo')
plt.show()
plt.plot(heart_array, 'go')
plt.show()

In [ ]:
# Feature engineering. Create more features based on the proceeding 5 minutes
def moving_average(a, n) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

gradient_average_30 = moving_average(gradient_array, 30)
gradient_average_120 = moving_average(gradient_array, 120)
gradient_average_240 = moving_average(gradient_array, 240)

speed_average_30 = moving_average(speed_array, 30)
speed_average_120  = moving_average(speed_array, 120)
speed_average_240 = moving_average(speed_array, 240)

In [ ]:
plt.plot(gradient_average_30, 'ro')
plt.show()
plt.plot(gradient_average_120, 'bo')
plt.show()
plt.plot(gradient_average_240, 'go')
plt.show()

In [ ]:
plt.plot(speed_average_30, 'ro')
plt.show()
plt.plot(speed_average_120, 'bo')
plt.show()
plt.plot(speed_average_240, 'go')
plt.show()

In [ ]:
half = len(heart_array)/2
heart_input_train = heart_array[:-half]
speed = speed_array[:-half]
speed_test = speed_array[-half:]
gradient_test= gradient_array[-half:]
actual_heart = heart_array[-half:]   
gradient_train = gradient_array[:-half]

In [ ]:

Train_speed = speed.reshape(-1,1)
Train_heart =heart_input_train.reshape(-1,1)
Train_gradient = gradient_train.reshape(-1,1)
Test_speed = speed_test.reshape(-1,1)
Actual_heart = actual_heart.reshape(-1,1)
Test_gradient = gradient_test.reshape(-1,1)

In [ ]:
#plt.plot(speed_array[-half:]  , heart_rate[-half:], 'bo')
#plt.plot(gradient_array[-half:]  , heart_rate[-half:], 'ro')
#plt.show()

In [ ]:
# Create linear regression object
reg = linear_model.LinearRegression()
reg.fit(Train_speed, Train_heart)

In [ ]:
# Make predictions using the testing set
heart_rate_predicted_regression = reg.predict(Test_speed)
#heart_rate_predicted_regression

In [ ]:
plt.plot(Test_speed ,heart_rate_predicted_regression, 'bo')
plt.plot(Test_speed ,Actual_heart, 'ro')
plt.show()

In [ ]:
# The coefficients
#print('Coefficients: \n', reg.coef_)

reg.coef_
reg.intercept_
# The mean squared error
#print("Mean squared error: %.2f"
#      % mean_squared_error(grad2, prediction_grad))
# Explained variance score: 1 is perfect prediction
#print('Variance score: %.2f' % r2_score(grad2, prediction_grad))

reg.score(Train_speed, Train_heart)

# should i be getting it between 0 and 1. get rid of the variation. can't remember what it's  called. 

In [ ]:
reg.intercept_

In [ ]:
# Now do it for the gradient instead of the speed
# Create linear regression object
reg_grad = linear_model.LinearRegression()
reg_grad.fit(Train_gradient, Train_heart)
heart_rate_predicted_on_gradient = reg_grad.predict(Test_gradient)

In [ ]:
plt.plot(Test_gradient ,heart_rate_predicted_on_gradient, 'bo')
plt.plot(Test_gradient ,Actual_heart, 'ro')
plt.show()

In [ ]:
reg_grad.score(Train_gradient, Train_heart)

In [ ]:
#Lets see if we can do a multiple linear regression that basically takes in both variables. 
# combine the speed and gradient data. 
combined_data= np.array([speed_array[0],gradient_array[0]])

for i in range(1,len(speed_array)):
    b= [speed_array[i], gradient_array[i]]
    combined_data = np.vstack((combined_data ,b)) 

half = len(combined_data)/2
    
train_input_data = combined_data[:-half]
test_input_data = combined_data[-half:]    

In [ ]:
len(train_input_data)
len(Train_heart)
train_input_data.shape
Train_heart.shape

In [ ]:
reg_two_linear = linear_model.LinearRegression()
reg_two_linear.fit(train_input_data, Train_heart)
heart_rate_predicted_on_both = reg_two_linear.predict(test_input_data)

In [ ]:
plt.plot(Test_speed ,heart_rate_predicted_on_both, 'bo')
plt.plot(Test_speed ,Actual_heart, 'ro')
plt.show()

In [ ]:
reg_two_linear.score(train_input_data, Train_heart)

In [ ]:
# a very tiny improvement from using both. Now i've a number of options that include using the engineered features and also none linear fits. 

In [ ]:
#SVR time
from sklearn.svm import SVR
clf = SVR(kernel = 'poly', epsilon=0.2)
#clf.fit(train_input_data, Train_heart) 
clf.fit(Train_speed, Train_heart)

In [ ]:
SVR_pre = clf.predict(Test_speed)

In [ ]:
plt.plot(Test_speed ,SVR_pre, 'bo')
plt.plot(Test_speed ,Actual_heart, 'ro')
plt.show()
clf.score(Train_speed, Train_heart)

In [ ]:
# what we could do is create this neatly to run through some of the different options. 
# now to get the SVR to take in both speed and gradient. 

In [ ]:
clf = SVR(kernel = 'poly', epsilon=0.2)
#clf.fit(train_input_data, Train_heart) 
clf.fit(train_input_data, Train_heart)

In [ ]:
SVR_pre = clf.predict(test_input_data)

In [ ]:
plt.plot(Test_speed ,SVR_pre, 'bo')
plt.plot(Test_speed ,Actual_heart, 'ro')
plt.show()
clf.score(train_input_data, Train_heart)

In [ ]:
# again small improvement . now i need to add in the engineered features. 